This script displays the locations of all DWD rain gauges on an interactive map.

In [ ]:
# import packages
import pandas as pd
from io import StringIO
import requests
import folium 

In [ ]:
# URL for the data
url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/precipitation/recent/RR_Stundenwerte_Beschreibung_Stationen.txt"

# Fetch the content from the URL
response = requests.get(url)
data = response.text

# Use StringIO to simulate a file-like object for pandas
data_io = StringIO(data)

# Read the first row as header and skip the second row, only read the first six columns into a DataFrame
df = pd.read_csv(data_io, sep="\s+", encoding='latin1', header=0, skiprows=[1], usecols=range(6))

# Convert "von_datum" and "bis_datum" to datetime format
df['von_datum'] = pd.to_datetime(df['von_datum'], format='%Y%m%d')
df['bis_datum'] = pd.to_datetime(df['bis_datum'], format='%Y%m%d')

# Display the DataFrame
print(df.head())


In [2]:
# check the data type
df.dtypes

Stations_id               int64
von_datum        datetime64[ns]
bis_datum        datetime64[ns]
Stationshoehe             int64
geoBreite               float64
geoLaenge               float64
dtype: object

In [3]:
# check for No-data
df.isna().sum()

Stations_id      0
von_datum        0
bis_datum        0
Stationshoehe    0
geoBreite        0
geoLaenge        0
dtype: int64

In [4]:
def add_marker_with_popup(m, location, popup_text, popup_width=120, popup_height=50):
    """
    Add a marker with a popup to a folium Map object.

    Parameters:
    - m (folium.Map): The folium Map object to add the marker to.
    - location (list): List containing latitude and longitude of the marker location.
    - popup_text (str): Text content to display in the popup.
    - popup_width (int): Width of the popup in pixels (default is 120).
    - popup_height (int): Height of the popup in pixels (default is 50).
    """
    popup_content = f"<div style='width: {popup_width}px; height: {popup_height}px;'>{popup_text}</div>"
    folium.Marker(
        location=location,
        popup=popup_content,
        icon=folium.Icon(color='black', icon='tint', prefix='fa')  # Custom rain gauge icon
    ).add_to(m)

# Example usage:
# Create a folium Map centered around Germany
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6, tiles='OpenStreetMap')

# Iterate through rows of a DataFrame 'df' and add markers with popups
for index, row in df.iterrows():
    popup_text = f"Station ID: {row['Stations_id']}<br>Von Datum: {row['von_datum'].strftime('%Y-%m-%d')}<br>Bis Datum: {row['bis_datum'].strftime('%Y-%m-%d')}"
    add_marker_with_popup(m, [row['geoBreite'], row['geoLaenge']], popup_text)

# Display the map
m
